In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [80]:
import networkx as nx
import math
import json
import matplotlib.pyplot as plt
import time


class OptimizedCoverTree:
    def __init__(self, nodes, distance_func):
        """
        Initialize the optimized cover tree.
        :param nodes: List of nodes in the graph or metric space.
        :param distance_func: A function that computes distances between nodes.
        """
        self.nodes = nodes
        self.distance_func = distance_func
        self.tree = self.build_cover_tree(nodes)
        self.edges = self.get_edges()

    def build_cover_tree(self, nodes):
        """
        Build the hierarchical cover tree using cluster-based optimization.
        :param nodes: List of nodes in the graph.
        :return: A dictionary representing levels of the tree.
        """
        levels = {}  # Dictionary to store levels of the tree
        radius = max(self.distance_func(u, v) for u in nodes for v in nodes if u != v)  # Initial radius
        parent_child_map = {}  # Map to track parent-child relationships
        level = 0

        while radius >= 1:  # Stop at the smallest meaningful radius
            uncovered_nodes = set(nodes)
            centers = []

            # Cluster nodes based on the radius
            while uncovered_nodes:
                # Select any uncovered node as a new center
                center = uncovered_nodes.pop()
                centers.append(center)

                # Remove all nodes within the current radius of this center
                uncovered_nodes -= {n for n in uncovered_nodes if self.distance_func(center, n) <= radius}

            levels[level] = centers

            # Establish parent-child connections
            if level > 0:  # Skip the first level (no parents at level -1)
                for child in centers:
                    # Find all potential parents from the previous level
                    parents = [
                        parent for parent in levels[level - 1]
                        if self.distance_func(parent, child) <= radius * 2
                    ]
                    parent_child_map[child] = parents

            radius /= 2  # Halve the radius
            level += 1

        self.parent_child_map = parent_child_map  # Store the parent-child relationships
        return levels

    def get_edges(self):
        """
        Retrieve edges to plot the cover tree.
        Each parent node connects to its children at the next level.
        """
        edges = []
        for child, parents in self.parent_child_map.items():
            for parent in parents:
                edges.append((parent, child))
        return edges


def compute_doubling_dimension(graph):
    """
    Compute the doubling dimension of a graph using an optimized cover tree.
    :param graph: The input graph (NetworkX graph).
    :return: Doubling dimension.
    """
    # Shortest-path metric
    lengths = dict(nx.all_pairs_dijkstra_path_length(graph))

    def graph_distance(u, v):
        return lengths[u][v]

    # Nodes of the graph
    nodes = list(graph.nodes)
    cover_tree = OptimizedCoverTree(nodes, graph_distance)

    # Calculate doubling constant and doubling dimension
    doubling_constant = max(
        sum(
            1
            for child in cover_tree.tree.get(level + 1, [])
            if graph_distance(parent, child) <= 2 ** level
        )
        for level, parents in cover_tree.tree.items()
        for parent in parents
    )
    doubling_dimension = math.log2(doubling_constant)

    return cover_tree, doubling_constant, round(doubling_dimension, 3)


In [82]:
def read_graph_from_json(filename):
    G = nx.Graph()
    with open(filename, "r") as f:
        data = json.load(f)  # Load the JSON file

    # Assuming 'inList' is the key in the JSON object
    inList = data[0]['inList']

    # Iterate through inList to create edges
    for target_node, sources in enumerate(inList):
        for source_node in sources:
            G.add_edge(source_node, target_node, weight=1)

    return G

def read_graph_from_txt(filename):
    G = nx.Graph()
    with open(filename, "r") as f:
        for line in f:
            node1, node2 = map(int, line.strip().split())  # Parse the two numbers
            G.add_edge(node1, node2, weight=1)  # Add an edge with weight 1
    return G


# File containing graph data for Twitter Dataset
json_file = "drive/MyDrive/Colab Notebooks/congress_network_data.json"

# File containing graph data for Facebook Dataset
# txt_file = "drive/MyDrive/Colab Notebooks/facebook.txt"

# Build the graph
G = read_graph_from_json(json_file)
# G = read_graph_from_txt(txt_file)

# Uncomment to Use Synthetic Data
# edges = [
#     ("A", "B", 1),
#     ("A", "C", 1),
#     ("A", "D", 1),
#     ("B", "E", 1),
#     ("B", "F", 1),
#     ("B", "G", 1),
#     ("C", "H", 1),
#     ("C", "I", 1),
#     ("D", "J", 1),
#     ("D", "K", 1),
#     ("E", "L", 1),
#     ("E", "M", 1),
#     ("F", "N", 1),
#     ("F", "O", 1),
#     ("G", "P", 1),
#     ("G", "Q", 1),
#     ("H", "R", 1),
#     ("H", "S", 1),
#     ("I", "T", 1),
#     ("I", "U", 1),
#     ("J", "V", 1),
#     ("J", "W", 1),
#     ("K", "X", 1),
#     ("K", "Y", 1),
#     ("L", "Z", 1),
#     ("M", "N", 1),
#     ("M", "P", 1),
#     ("N", "Q", 1),
#     ("O", "R", 1),
#     ("O", "T", 1),
#     ("P", "U", 1),
#     ("Q", "V", 1),
#     ("R", "W", 1),
#     ("S", "X", 1),
#     ("T", "Y", 1),
#     ("U", "Z", 1),
#     ("V", "M", 1),
#     ("W", "O", 1),
#     ("X", "P", 1),
#     ("Y", "R", 1),
#     ("Z", "T", 1)
# ]


# Build the graph for Synthetic Data
# G = nx.Graph()
# G.add_weighted_edges_from(edges)

In [83]:
# Visualize the graph
def visualize_graph(G):
    pos = nx.spring_layout(G, seed=10)  # Seed ensures consistent layout
    plt.figure(figsize=(8, 6))
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_color="lightblue",
        edge_color="gray",
        node_size=300,
        font_size=8,
        font_weight="bold",
    )
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=10)
    plt.title("Graph Visualization with Edge Weights", fontsize=14)
    plt.show()

#Uncomment the Below Line to Visualize the Graph
# visualize_graph(G)

In [84]:
if nx.is_connected(G):
    print("The graph is connected.")
else:
    print("The graph is not connected.")


The graph is connected.


In [85]:
start_time = time.time()

# Compute the doubling dimension using the optimized cover tree
cover_tree, doubling_constant, doubling_dimension = compute_doubling_dimension(G)

end_time = time.time()

# Display results
print("Optimized Cover Tree Levels:")
for level, nodes in cover_tree.tree.items():
    print(f"Level {level}: {nodes}")

print("\nDoubling Constant:", doubling_constant)
print("Doubling Dimension:", doubling_dimension)

execution_time = end_time - start_time
print(f"\nExecution Time: {execution_time:.6f} seconds")


Optimized Cover Tree Levels:
Level 0: [0]
Level 1: [0, 34, 100, 196, 285, 297, 395, 404]
Level 2: [0, 1, 2, 3, 5, 6, 7, 10, 14, 16, 34, 48, 63, 75, 78, 93, 94, 95, 96, 97, 98, 99, 100, 101, 107, 114, 121, 123, 127, 131, 133, 144, 146, 153, 163, 166, 167, 172, 173, 177, 187, 196, 209, 210, 216, 224, 225, 227, 230, 240, 252, 265, 270, 277, 280, 282, 287, 309, 314, 316, 334, 342, 344, 362, 378, 384, 395, 404, 414, 434, 458, 465]

Doubling Constant: 47
Doubling Dimension: 5.555

Execution Time: 7.080956 seconds
